# How To Export a Model Trained in Supervisely

After you have trained a model using Supervisely application, you can download its weights and config file, and use the model outside Supervisely Platform.

## 1. Download model weights and config from Team Files

Go to Team Files and download model weights and config. They are located in `/mmdetection3d-v1.x`

![Team Files](img/download_mmdet3d.png)

**Alternatively, you can download these files using Supervisely API:**

In [ ]:
# Install supervisely SDK if not installed
!pip install supervisely

In [ ]:
# Fill in these arguments:
server_address = "https://..."
api_token = "abcd0123456789..."
team_id = ...
path_to_weights_in_team_files = ...
path_to_config_in_team_files = ...
download_dir = "./my_model"

import supervisely as sly
api = sly.Api(server_address, api_token)
api.file.download(team_id, path_to_weights_in_team_files, f"{download_dir}/weights.pth")
api.file.download(team_id, path_to_config_in_team_files, f"{download_dir}/config.py")

## 2. Install requirements

**You can use our pre-builded docker image:**

`supervisely/mmdet3d-v1.x:1.0.1`

Or build the image using this [Dockerfile](docker/mmdet3d-v1.x.Dockerfile).

**Alternatively, you can install only needed requirements with pip:**

In [ ]:
!pip install torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu118
!pip install -U openmim
!mim install mmengine 'mmcv>=2.0.0rc4' 'mmdet>=3.0.0' 'mmdet3d>=1.1.0'

# To convert .pcd to .bin
!pip install git+https://github.com/DanielPollithy/pypcd.git

# Boost GPU perfomance (optional)
!pip install spconv-cu118 cumm-cu118

# Fix issue with open3d
!pip install Werkzeug==2.2.3
!pip install numpy==1.22.0

## 3. Build the model

After you've installed requirements and downloaded model weights and config, **fill in the paths** where the files located:

In [8]:
# Insert your paths here:
weights_path = "my_model/epoch_20.pth"
config_path = "my_model/config.py"

In [2]:
from typing import Optional, Union, List
from mmdet3d.apis import LidarDet3DInferencer
from mmdet3d.utils import ConfigType
from mmdet3d.registry import DATASETS, TRANSFORMS
from mmdet3d.datasets.det3d_dataset import Det3DDataset
from mmdet3d.datasets.transforms import LoadPointsFromFile
from mmdet3d.structures.bbox_3d import get_box_type
from mmengine.dataset import Compose
from mmcv.transforms.base import BaseTransform
import open3d as o3d
import numpy as np


def up_bbox3d(bbox3d: list):
    # z += h / 2
    bbox3d = bbox3d.copy()
    bbox3d[2] += bbox3d[5] / 2
    return bbox3d


@TRANSFORMS.register_module()
class LoadPointsFromPcdFile(LoadPointsFromFile):
    
    def __init__(self,
                 coord_type: str,
                 load_dim: int = 6,
                 use_dim: Union[int, List[int]] = [0, 1, 2],
                 shift_height: bool = False,
                 use_color: bool = False,
                 norm_intensity: bool = False,
                 norm_elongation: bool = False,
                 backend_args: Optional[dict] = None,
                 zero_aux_dims: bool = False
                 ) -> None:
        super().__init__(coord_type, load_dim, use_dim, shift_height, use_color, norm_intensity, norm_elongation, backend_args)
        self.zero_aux_dims = zero_aux_dims

    def _load_points(self, pts_filename: str) -> np.ndarray:
        pcd = o3d.io.read_point_cloud(pts_filename)
        xyz = np.asarray(pcd.points, dtype=np.float32)
        if self.load_dim > 3:
            aux_dims = self.load_dim - 3
            if pcd.has_colors() and not self.zero_aux_dims:
                rgb = np.asarray(pcd.colors, dtype=np.float32)
            else:
                rgb = np.zeros((xyz.shape[0], aux_dims), dtype=np.float32)
            points = np.concatenate([xyz, rgb[:, :aux_dims]], 1)
        else:
            points = xyz
        return points


@TRANSFORMS.register_module()
class PCDLoader(BaseTransform):
    """Load point cloud in the Inferencer's pipeline.

    Added keys:
      - points
      - timestamp
      - axis_align_matrix
      - box_type_3d
      - box_mode_3d
    """

    def __init__(self, coord_type='LIDAR', zero_aux_dims: bool = False, **kwargs) -> None:
        super().__init__()
        self.from_file = TRANSFORMS.build(
            dict(type='LoadPointsFromPcdFile', coord_type=coord_type, zero_aux_dims=zero_aux_dims, **kwargs))
        self.from_ndarray = TRANSFORMS.build(
            dict(type='LoadPointsFromDict', coord_type=coord_type, **kwargs))
        self.box_type_3d, self.box_mode_3d = get_box_type(coord_type)

    def transform(self, single_input: dict) -> dict:
        """Transform function to add image meta information.
        Args:
            single_input (dict): Single input.

        Returns:
            dict: The dict contains loaded image and meta information.
        """
        assert 'points' in single_input, "key 'points' must be in input dict"
        if isinstance(single_input['points'], str):
            inputs = dict(
                lidar_points=dict(lidar_path=single_input['points']),
                timestamp=1,
                # for ScanNet demo we need axis_align_matrix
                axis_align_matrix=np.eye(4),
                box_type_3d=self.box_type_3d,
                box_mode_3d=self.box_mode_3d)
        elif isinstance(single_input['points'], np.ndarray):
            inputs = dict(
                points=single_input['points'],
                timestamp=1,
                # for ScanNet demo we need axis_align_matrix
                axis_align_matrix=np.eye(4),
                box_type_3d=self.box_type_3d,
                box_mode_3d=self.box_mode_3d)
        else:
            raise ValueError('Unsupported input points type: '
                             f"{type(single_input['points'])}")

        if 'points' in inputs:
            return self.from_ndarray(inputs)
        return self.from_file(inputs)


class PcdDet3DInferencer(LidarDet3DInferencer):

    def __init__(self,
                 model: Union[str, None] = None,
                 weights: Optional[str] = None,
                 device: Optional[str] = None,
                 zero_aux_dims: bool = False,
                 scope: str = 'mmdet3d',
                 palette: str = 'none',
                 ) -> None:
        self.zero_aux_dims = zero_aux_dims
        super().__init__(
            model=model,
            weights=weights,
            device=device,
            scope=scope,
            palette=palette)

    def _init_pipeline(self, cfg: ConfigType) -> Compose:
        """Initialize the test pipeline."""
        pipeline_cfg = cfg.test_dataloader.dataset.pipeline

        load_point_idx = self._get_transform_idx(pipeline_cfg, 'LoadPointsFromFile')
        if load_point_idx == -1:
            load_point_idx = self._get_transform_idx(pipeline_cfg, 'LoadPointsFromPcdFile')
        
        if load_point_idx == -1:
            raise ValueError(
                'LoadPointsFromFile/LoadPointsFromPcdFile is not found in the test pipeline')

        load_cfg = pipeline_cfg[load_point_idx]
        self.coord_type, self.load_dim = load_cfg['coord_type'], load_cfg[
            'load_dim']
        self.use_dim = list(range(load_cfg['use_dim'])) if isinstance(
            load_cfg['use_dim'], int) else load_cfg['use_dim']

        pipeline_cfg[load_point_idx]['type'] = 'PCDLoader'
        pipeline_cfg[load_point_idx]['zero_aux_dims'] = self.zero_aux_dims
        return Compose(pipeline_cfg)

    def _init_visualizer(self, cfg: ConfigType):
        return None
    

@DATASETS.register_module()
class CustomDataset(Det3DDataset):
    def __init__(self, *args, **kwargs):
        self.METAINFO = {"classes": kwargs["selected_classes"], "palette": []}
        self._metainfo = self.METAINFO
        # self.METAINFO = {"classes": kwargs["selected_classes"], "palette": []}
        # super().__init__(*args, **kwargs)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
from mmengine import Config

device = "cuda:0"
cfg = Config.fromfile(config_path)
classes = cfg.train_dataloader.dataset.selected_classes
model = PcdDet3DInferencer(cfg, weights_path, device)

Loads checkpoint by local backend from path: my_model/epoch_20.pth
03/14 14:29:34 - mmengine - WARNING - Failed to search registry with scope "mmdet3d" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet3d" is a correct scope, or whether the registry is initialized.


## 4. Inference

In [23]:
pcd_path = "tmp.pcd"
conf_thresh = 0.4

results_dict = model(inputs=dict(points=pcd_path), no_save_vis=True)

predictions = results_dict["predictions"][0]
bboxes_3d = predictions["bboxes_3d"]
labels_3d = predictions["labels_3d"]
scores_3d = predictions["scores_3d"]

result = []
for bbox3d, label3d, score3d in zip(bboxes_3d, labels_3d, scores_3d):
    if score3d < conf_thresh:
        continue
    result.append({
        "bbox3d": up_bbox3d(bbox3d),  # [x, y, z, w, l, h, rot_z]
        "label": classes[label3d],
        "score": score3d
    })

Output()

In [25]:
result

[{'bbox3d': [13.755786895751953,
   2.1196670532226562,
   -0.8083688616752625,
   3.9115381240844727,
   2.611595392227173,
   1.5770412683486938,
   -2.5685808658599854,
   3.72103725609918e-09,
   4.733932135181362e-10],
  'label': 'Car',
  'score': 0.4739571809768677},
 {'bbox3d': [29.394603729248047,
   8.114307403564453,
   -1.0389283299446106,
   3.4990007877349854,
   1.987350583076477,
   1.35783851146698,
   -2.474350929260254,
   3.72103725609918e-09,
   4.733932135181362e-10],
  'label': 'Car',
  'score': 0.44827646017074585},
 {'bbox3d': [7.485008239746094,
   -0.120880126953125,
   -1.2826476097106934,
   3.7444984912872314,
   2.1186046600341797,
   1.1602580547332764,
   -2.5160534381866455,
   -0.0019796208944171667,
   0.0009996920125558972],
  'label': 'Car',
  'score': 0.40709176659584045}]